In [0]:
!pip install tf-nightly-2.0-preview

In [0]:

import tensorflow as tf

Limited tf.compat.v2.summary API due to missing TensorBoard installation
Limited tf.summary API due to missing TensorBoard installation


In [0]:

print("You have version", tf.__version__)
assert tf.__version__ >= "2.0" # TensorFlow ≥ 2.0 required

You have version 2.0.0-dev20190214


In [85]:
import numpy as np

from __future__ import absolute_import, division, print_function

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

print(tf.__version__)

from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Flatten

2.0.0-dev20190214


In [0]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

train_images = train_images.astype('float32')
test_images = test_images.astype('float32')

train_images = train_images / 255
test_images = test_images / 255


In [0]:
BATCH_SIZE = 128
BUFFER_SIZE = len(train_images)

mnist_train = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
mnist_test = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(BATCH_SIZE)

In [0]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10, activation='softmax')

  def call(self, x):
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)
  
model = MyModel()

In [0]:
loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
optimizer = tf.keras.optimizers.Adam()

In [0]:

train_loss_metric = tf.keras.metrics.Mean(name='train_loss')
test_loss_metric = tf.keras.metrics.Mean(name='test_loss')

train_accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [0]:
def train_step(images, labels):
  with tf.GradientTape() as tape:    
    # Forward pass
    predictions = model(images)
    train_loss = loss_function(y_true=labels, y_pred=predictions)
    
  
  # Backward pass
  gradients = tape.gradient(train_loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  # Record results
  train_loss_metric(train_loss)
  train_accuracy_metric(labels, predictions)
 

In [0]:
def test_step(images, labels):
  predictions = model(images)
  test_loss = loss_function(y_true=labels, y_pred=predictions)
  
  # Record results
  test_loss_metric(test_loss)
  test_accuracy_metric(labels, predictions)

In [94]:
EPOCHS = 5
train_loss_metric_array=[]
train_accuracy_metric_array=[]


for epoch in range(EPOCHS):
  for images, labels in mnist_train:
    train_step(images, labels)
    #train_loss_metric_array.append(train_loss_metric.result().numpy)
  
  for test_images, test_labels in mnist_test:
    test_step(test_images, test_labels)
    #train_accuracy_metric_array.append(test_accuracy_metric.result().numpy)
  
  template = 'Epoch {}, Loss: {:.4f}, Accuracy: {:.2f}, Test loss: {:.4f}, Test accuracy: {:.2f}'
  print (template.format(epoch +1, 
                         train_loss_metric.result(),
                         train_accuracy_metric.result() * 100, 
                         test_loss_metric.result(), 
                         test_accuracy_metric.result() * 100))

Epoch 1, Loss: 0.5414, Accuracy: 81.63, Test loss: 1.4229, Test accuracy: 51.51
Epoch 2, Loss: 0.4710, Accuracy: 83.80, Test loss: 1.0919, Test accuracy: 62.57
Epoch 3, Loss: 0.4347, Accuracy: 84.89, Test loss: 0.9188, Test accuracy: 68.37
Epoch 4, Loss: 0.4102, Accuracy: 85.67, Test loss: 0.8112, Test accuracy: 72.02
Epoch 5, Loss: 0.3917, Accuracy: 86.26, Test loss: 0.7362, Test accuracy: 74.51


Deep model

In [0]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.d3 = Dense(256, activation='relu')
    self.d4 = Dense(128, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(256, activation='relu')
    self.d2 = Dense(10, activation='softmax')

  def call(self, x):
    x = self.d3(x)
    x = self.d4(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)
  
model = MyModel()

In [0]:
loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
optimizer = tf.keras.optimizers.Adam()

In [0]:
train_loss_metric = tf.keras.metrics.Mean(name='train_loss')
test_loss_metric = tf.keras.metrics.Mean(name='test_loss')

train_accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [0]:
def train_step(images, labels):
  with tf.GradientTape() as tape:    
    # Forward pass
    predictions = model(images)
    train_loss = loss_function(y_true=labels, y_pred=predictions)
    
  
  # Backward pass
  gradients = tape.gradient(train_loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  # Record results
  train_loss_metric(train_loss)
  train_accuracy_metric(labels, predictions)

In [0]:
def test_step(images, labels):
  predictions = model(images)
  test_loss = loss_function(y_true=labels, y_pred=predictions)
  
  # Record results
  test_loss_metric(test_loss)
  test_accuracy_metric(labels, predictions)

In [106]:
EPOCHS = 5

for epoch in range(EPOCHS):
  for images, labels in mnist_train:
    train_step(images, labels)
  
  for test_images, test_labels in mnist_test:
    test_step(test_images, test_labels)
  
  template = 'Epoch {}, Loss: {:.4f}, Accuracy: {:.2f}, Test loss: {:.4f}, Test accuracy: {:.2f}'
  print (template.format(epoch +1, 
                         train_loss_metric.result(), 
                         train_accuracy_metric.result() * 100, 
                         test_loss_metric.result(), 
                         test_accuracy_metric.result() * 100))

Epoch 1, Loss: 0.5491, Accuracy: 81.09, Test loss: 0.4405, Test accuracy: 84.48
Epoch 2, Loss: 0.4738, Accuracy: 83.60, Test loss: 0.4186, Test accuracy: 85.21
Epoch 3, Loss: 0.4352, Accuracy: 84.84, Test loss: 0.4050, Test accuracy: 85.81
Epoch 4, Loss: 0.4093, Accuracy: 85.66, Test loss: 0.3970, Test accuracy: 86.06
Epoch 5, Loss: 0.3898, Accuracy: 86.27, Test loss: 0.3912, Test accuracy: 86.24
